<a href="https://colab.research.google.com/github/Sonlux/BL_eth_fe/blob/main/Intelligent%20Customer%20Behavior%20Prediction%20for%20E-commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install Kaggle API
!pip install kaggle

# Make a Kaggle directory and move the kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
# Replace 'USERNAME/DATASET_NAME' with the actual dataset path
!kaggle datasets download -d zeesolver/consumer-behavior-and-shopping-habits-dataset

# Unzip the downloaded dataset (if needed)
!unzip consumer-behavior-and-shopping-habits-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/zeesolver/consumer-behavior-and-shopping-habits-dataset
License(s): CC0-1.0
100% 146k/146k [00:00<00:00, 473kB/s]
100% 146k/146k [00:00<00:00, 472kB/s]
Archive:  consumer-behavior-and-shopping-habits-dataset.zip
  inflating: shopping_behavior_updated.csv  
  inflating: shopping_trends.csv     


In [5]:
import pandas as pd

# Load the CSV file (replace 'your_dataset.csv' with the actual file name)
data = pd.read_csv('shopping_behavior_updated.csv')
data.head()


,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [6]:
# Check for missing values
print(data.isnull().sum())

# Drop or fill missing values
data = data.dropna()  # or you can fill with mean/median depending on your data

# Basic feature engineering (e.g., convert categorical variables to numerical)
data = pd.get_dummies(data, columns=['Category', 'Gender'], drop_first=True)


Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64


In [7]:
print(data.columns)


Index(['Customer ID', 'Age', 'Item Purchased', 'Purchase Amount (USD)',
       'Location', 'Size', 'Color', 'Season', 'Review Rating',
       'Subscription Status', 'Shipping Type', 'Discount Applied',
       'Promo Code Used', 'Previous Purchases', 'Payment Method',
       'Frequency of Purchases', 'Category_Clothing', 'Category_Footwear',
       'Category_Outerwear', 'Gender_Male'],
      dtype='object')


In [8]:
# Convert columns to numeric, if they aren't already
data['Purchase Amount (USD)'] = pd.to_numeric(data['Purchase Amount (USD)'], errors='coerce')
data['Frequency of Purchases'] = pd.to_numeric(data['Frequency of Purchases'], errors='coerce')

# Now calculate average purchase amount
data['avg_purchase'] = data['Purchase Amount (USD)'] / data['Frequency of Purchases']


In [9]:
# Step 4.1: Handle missing values
data = data.fillna(0)  # Replace NaN with 0, or use another strategy

# Step 4.2: Encode categorical variables
data = pd.get_dummies(data, columns=['Location', 'Size', 'Color', 'Season', 'Payment Method'], drop_first=True)

# Step 4.3: Feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = ['Purchase Amount (USD)', 'Frequency of Purchases', 'avg_purchase']
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Check data after preprocessing
data.head()


,Customer ID,Age,Item Purchased,Purchase Amount (USD),Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,...,Color_White,Color_Yellow,Season_Spring,Season_Summer,Season_Winter,Payment Method_Cash,Payment Method_Credit Card,Payment Method_Debit Card,Payment Method_PayPal,Payment Method_Venmo
0,1,55,Blouse,-0.285629,3.1,Yes,Express,Yes,Yes,14,...,False,False,False,False,True,False,False,False,False,True
1,2,19,Sweater,0.178852,3.1,Yes,Express,Yes,Yes,2,...,False,False,False,False,True,True,False,False,False,False
2,3,50,Jeans,0.558882,3.1,Yes,Free Shipping,Yes,Yes,23,...,False,False,True,False,False,False,True,False,False,False
3,4,21,Sandals,1.276716,3.5,Yes,Next Day Air,Yes,Yes,49,...,False,False,True,False,False,False,False,False,True,False
4,5,45,Blouse,-0.454531,2.7,Yes,Free Shipping,Yes,Yes,31,...,False,False,True,False,False,False,False,False,True,False


In [10]:
from sklearn.model_selection import train_test_split

# Define the target variable (assuming it’s a column like 'Churn' or 'High-Value Customer')
X = data.drop(columns=['Customer ID', 'Age'])  # Replace 'TargetColumn' with your actual target column name
y = data['Age']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
# Step 1: Encode categorical features
data_encoded = pd.get_dummies(data, drop_first=True)

# Step 2: Redefine X and y with the encoded dataset
X = data_encoded.drop(columns=['Customer ID', 'Age'])  # Replace 'Churn' with your target column
y = data['Age']  # Target column (replace with your actual target column)

# Step 3: Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Initialize and train the model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [13]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.017948717948717947

Classification Report:
               precision    recall  f1-score   support

          18       0.07      0.04      0.05        27
          19       0.00      0.00      0.00        18
          20       0.00      0.00      0.00        18
          21       0.09      0.09      0.09        23
          22       0.00      0.00      0.00        18
          23       0.00      0.00      0.00        17
          24       0.00      0.00      0.00        15
          25       0.02      0.03      0.03        29
          26       0.00      0.00      0.00        24
          27       0.00      0.00      0.00        23
          28       0.03      0.04      0.04        27
          29       0.08      0.08      0.08        25
          30       0.04      0.05      0.04        22
          31       0.00      0.00      0.00        25
          32       0.03      0.04      0.03        27
          33       0.00      0.00      0.00        22
          34       0.08  

In [14]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
}

# Grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Use the best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': 10, 'n_estimators': 100}


In [15]:
import joblib

# Save the model
joblib.dump(best_model, 'customer_behavior_model.pkl')

# To load and use the model later
loaded_model = joblib.load('customer_behavior_model.pkl')


In [16]:
# Sample data for prediction
sample_data = X_test.iloc[0:1]  # Replace with actual new data

# Predict using the loaded model
prediction = loaded_model.predict(sample_data)
print("Predicted Class:", prediction)


Predicted Class: [24]
